In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/mzhang3/Data/NSAS_Bench201/lib'))
sys.path.append(str('/home/mzhang3/Data/NSAS_Bench201/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

In [2]:
def _concat(xs):
    return torch.cat([x.view(-1) for x in xs])


def _hessian_vector_product(vector, network, criterion, xloader, eta, r=1e-2):
    
    base_inputs, base_targets, _, _= next(iter(xloader))########only use one batch for stochatistic    

    
    p_model = deepcopy(network)       
    base_targets = base_targets.cuda(non_blocking=True)        
    n = base_inputs.size(0)         
    _, logits = p_model(base_inputs)      
    loss = criterion(logits, base_targets)      
    grads = torch.autograd.grad(loss, p_model.module.get_weights(),create_graph=True)   
    prod=sum([(g*v).sum() for g,v in zip(grads, vector)]) 
    prod.backward()        
    
    
    grads_1=[v-eta*p.grad.data for v,p in zip(vector, p_model.module.get_weights())] 
    
    del p_model
    p_model = deepcopy(network)       
    base_targets = base_targets.cuda(non_blocking=True)        
    n = base_inputs.size(0)         
    _, logits = p_model(base_inputs)      
    loss = criterion(logits, base_targets)      
    grads = torch.autograd.grad(loss, p_model.module.get_weights(),create_graph=True)   
    prod=sum([(g*v).sum() for g,v in zip(grads, grads_1)]) 
    prod.backward()     
    
    grads_2=[v-eta*p.grad.data for v,p in zip(grads_1, p_model.module.get_weights())] 
    
    grads_sum=[v+p+g for v,p,g in zip(vector, grads_1, grads_2)] 
    
    
    
    R = r / _concat(vector).norm()
    for p, g in zip(network.module.get_weights(), grads_sum):
        p.data.add_(R, g)    


    
    base_targets = base_targets.cuda(non_blocking=True)        
    n = base_inputs.size(0)         
    _, logits = network(base_inputs)      
    loss = criterion(logits, base_targets)      
    grads_p = torch.autograd.grad(loss, network.module.get_alphas())

    for p, g in zip(network.module.get_weights(), grads_sum):
        p.data.sub_(2*R, g) 
              
    base_targets = base_targets.cuda(non_blocking=True)
    n = base_inputs.size(0)         
    _, logits = network(base_inputs) 
    loss = criterion(logits, base_targets)
    grads_n = torch.autograd.grad(loss, network.module.get_alphas())

    for p, v in zip(network.module.get_weights(), vector):  ####We empirically found that adding noise into supernet could improve the performance
        p.data.add_(R, v)
    return [(x-y).div_(2*R) for x, y in zip(grads_p, grads_n)]


def backward_step_unrolled(network, criterion, xloader, w_optimizer, arch_inputs, arch_targets):

    LR, WD, momentum = w_optimizer.param_groups[0]['lr'], w_optimizer.param_groups[0]['weight_decay'], w_optimizer.param_groups[0]['momentum']
    network_optimizer = deepcopy(w_optimizer)
    eta=LR
    #for step, (input, target) in enumerate(train_queue):
    for step in range(4):####################how many steps
        base_inputs, base_targets, _, _ = next(iter(xloader))
        base_targets = base_targets.cuda(non_blocking=True)

        n = base_inputs.size(0)
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        loss = criterion(logits, base_targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(network.module.get_weights(), 5)
        network_optimizer.step()    

        
    unrolled_model = deepcopy(network)
    unrolled_model.zero_grad()
    _, unrolled_logits = unrolled_model(arch_inputs)
    unrolled_loss = criterion(unrolled_logits, arch_targets)
    unrolled_loss.backward()

    dalpha = unrolled_model.module.arch_parameters.grad
    vector = [v.grad.data for v in unrolled_model.module.get_weights()]
    
    
    
    [implicit_grads] = _hessian_vector_product(vector, network, criterion, xloader,eta)

    dalpha.data.sub_(LR, implicit_grads.data)

    if network.module.arch_parameters.grad is None:
        network.module.arch_parameters.grad = deepcopy( dalpha )
    else:
        network.module.arch_parameters.grad.data.copy_( dalpha.data )
        
    del unrolled_model     
    return unrolled_loss.detach(), unrolled_logits.detach()


In [3]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the architecture-weight
        a_optimizer.zero_grad()
        arch_loss, arch_logits = backward_step_unrolled(network, criterion, xloader, w_optimizer, arch_inputs, arch_targets)
        a_optimizer.step()      
        
        # record
        arch_prec1, arch_prec5 = obtain_accuracy(arch_logits.data, arch_targets.data, topk=(1, 5))
        arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
        arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
        arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))


        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Astr = 'Arch [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=arch_losses, top1=arch_top1, top5=arch_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Astr)
            
            genotype=network.module.genotype()
            logger.log(genotype)
    return arch_losses, arch_top1.avg, arch_top5.avg

def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
            arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
            arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg


In [4]:
parser = argparse.ArgumentParser("DARTS Second Order")
parser.add_argument('--data_path',          type=str,   default= '/data/mzhang3/ENNAS_benchmark102/data/cifar-10-batches-py', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
#parser.add_argument('--track_running_stats',type=int,   default= 1,choices=[0,1],help='Whether use track_running_stats or not in the BN layer.')
#parser.add_argument('--config_path',        type=str,   default= '/data/jiahzhao/NSAS_Bench201/configs/nas-benchmark/algos/GDAS.config'  ,help='The path of the configuration.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
#parser.add_argument('--tau_min',            type=float, default=0.1, help='The minimum tau for Gumbel')
#parser.add_argument('--tau_max',            type=float, default=10, help='The maximum tau for Gumbel')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./T4_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/mzhang3/Data/NSAS_Bench201/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=100,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0, help='manual seed')
args = parser.parse_args([])


xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/mzhang3/Data/NSAS_Bench201/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config_path = '/home/mzhang3/Data/NSAS_Bench201/configs/nas-benchmark/algos/DARTS_wlr0p01.config'
config = load_config(config_path, {'class_num': class_num, 'xshape': xshape}, logger)

###config.batch_size

# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'DARTS-V2', 'C': xargs.channel, 'N': xargs.num_cells,
                              'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                              'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)
logger.log('search-model :\n{:}'.format(search_model))

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)
#a_optimizer = Vadam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), prec_init =1, train_set_size=25000, num_samples=1)


logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    a_optimizer.load_state_dict ( checkpoint['a_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

# start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    min_LR    = min(w_scheduler.get_lr())
    logger.log('\n[Search the {:}-th epoch] {:}, LR={:}'.format(epoch_str, need_time, min_LR))

    search_w_loss, search_w_top1, search_w_top5 = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, epoch_str, xargs.print_freq, logger)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss.avg, search_w_top1, search_w_top5, search_time.sum))
    valid_a_loss , valid_a_top1 , valid_a_top5  = valid_func(valid_loader, network, criterion)
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss, valid_a_top1, valid_a_top5))
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        genotypes['best']        = search_model.genotype()
        find_best = True
    else: find_best = False

    genotypes[epoch] = search_model.genotype()
    logger.log('<<<--->>> The {:}-th epoch : {:}'.format(epoch_str, genotypes[epoch]))
    # save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
                'args'  : deepcopy(xargs),
                'search_model': search_model.state_dict(),
                'w_optimizer' : w_optimizer.state_dict(),
                'a_optimizer' : a_optimizer.state_dict(),
                'w_scheduler' : w_scheduler.state_dict(),
                'genotypes'   : genotypes,
                'valid_accuracies' : valid_accuracies},
                model_base_path, logger)
    last_info = save_checkpoint({
          'epoch': epoch + 1,
          'args' : deepcopy(args),
          'last_checkpoint': save_path,
          }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    with torch.no_grad():
        logger.log('arch-parameters :\n{:}'.format( nn.functional.softmax(search_model.arch_parameters, dim=-1).cpu() ))
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*100)
# check the performance from the architecture dataset
logger.log('DARTS-V2 : run {:} epochs, cost {:.1f} s, last-geno is {:}.'.format(total_epoch, search_time.sum, genotypes[total_epoch-1]))
if api is not None: logger.log('{:}'.format( api.query_by_arch(genotypes[total_epoch-1]) ))
logger.close()

Main Function with logger : Logger(dir=T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.0003
arch_nas_dataset : /home/mzhang3/Data/NSAS_Bench201/NAS-Bench-102-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
data_path        : /data/mzhang3/ENNAS_benchmark102/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 100
rand_seed        : 0
save_dir         : ./T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
workers          : 2
Python  Version  : 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34)  [GCC 7.3.0]
Pillow  Version  : 5.4.1
PyTorch Version  : 1.6.0
cuDNN   Version  : 7605
CUDA available   : True
CUDA GPU numbers : 1
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
Load split file from /home/mzhang3/Dat

[2021-02-07 12:17:31] create API = NASBench102API(15625/15625 architectures) done
=> do not find the last-info file : T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth

[Search the 000-030-th epoch] Time Left: [00:00:00], LR=0.010000000000000002


/data/mzhang3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)


*SEARCH* [2021-02-07 12:17:37] [000-030][000/391] Time 5.30 (5.30) Data 0.29 (0.29) Arch [Loss 2.326 (2.326)  Prec@1 10.94 (10.94) Prec@5 51.56 (51.56)]
Structure(4 nodes with |nor_conv_3x3~0|+|skip_connect~0|none~1|+|none~0|skip_connect~1|nor_conv_1x1~2|)
*SEARCH* [2021-02-07 12:28:47] [000-030][100/391] Time 6.93 (6.69) Data 0.00 (0.00) Arch [Loss 1.999 (2.203)  Prec@1 25.00 (17.00) Prec@5 85.94 (65.47)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|avg_pool_3x3~0|nor_conv_3x3~1|skip_connect~2|)
*SEARCH* [2021-02-07 12:40:46] [000-030][200/391] Time 7.25 (6.94) Data 0.00 (0.00) Arch [Loss 1.834 (2.103)  Prec@1 28.12 (21.47) Prec@5 92.19 (73.68)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|avg_pool_3x3~0|nor_conv_3x3~1|avg_pool_3x3~2|)
*SEARCH* [2021-02-07 12:52:45] [000-030][300/391] Time 7.29 (7.02) Data 0.00 (0.00) Arch [Loss 1.889 (2.038)  Prec@1 29.69 (23.74) Prec@5 84.38 (77.12)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x

*SEARCH* [2021-02-07 14:39:27] [002-030][390/391] Time 6.97 (7.26) Data 0.00 (0.00) Arch [Loss 1.506 (1.641)  Prec@1 45.00 (39.80) Prec@5 90.00 (89.59)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|none~2|)
[002-030] searching : loss=1.64, accuracy@1=39.80%, accuracy@5=89.59%, time-cost=8478.0 s
[002-030] evaluate  : loss=1.54, accuracy@1=43.62%, accuracy@5=90.93%
<<<--->>> The 002-030-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|none~2|)
Find T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into T1_K2_hess_seed0/output/search-cell-nas-bench-1

...

*SEARCH* [2021-02-08 03:31:51] [019-030][000/391] Time 6.73 (6.73) Data 0.29 (0.29) Arch [Loss 1.127 (1.127)  Prec@1 56.25 (56.25) Prec@5 93.75 (93.75)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 03:44:04] [019-030][100/391] Time 7.47 (7.32) Data 0.00 (0.00) Arch [Loss 1.296 (1.081)  Prec@1 45.31 (62.38) Prec@5 95.31 (95.76)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 03:56:35] [019-030][200/391] Time 7.43 (7.41) Data 0.00 (0.00) Arch [Loss 1.079 (1.060)  Prec@1 60.94 (62.93) Prec@5 96.88 (96.21)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 04:08:52] [019-030][300/391] Time 5.97 (7.40) Data 0.00 (0.00) Arch [Loss 1.011 (1.059)  Prec@1 70.31 (62.81) Prec@5 95.31 (96.20)]
Structure(4 nodes with |nor_

*SEARCH* [2021-02-08 05:33:44] [021-030][200/391] Time 7.56 (7.39) Data 0.00 (0.00) Arch [Loss 1.166 (1.022)  Prec@1 57.81 (64.15) Prec@5 90.62 (96.35)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 05:45:59] [021-030][300/391] Time 6.13 (7.38) Data 0.00 (0.00) Arch [Loss 0.934 (1.025)  Prec@1 71.88 (64.16) Prec@5 96.88 (96.23)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 05:57:03] [021-030][390/391] Time 7.75 (7.38) Data 0.00 (0.00) Arch [Loss 1.004 (1.018)  Prec@1 62.50 (64.39) Prec@5 92.50 (96.32)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
[021-030] searching : loss=1.02, accuracy@1=64.39%, accuracy@5=96.32%, time-cost=63188.3 s
[021-030] evaluate  : loss=0.96, accuracy@1=66.20%, accuracy@5=96.98%
<<<--->>> The 021-03

*SEARCH* [2021-02-08 07:34:59] [023-030][390/391] Time 7.75 (7.54) Data 0.00 (0.00) Arch [Loss 1.009 (0.984)  Prec@1 70.00 (65.71) Prec@5 97.50 (96.79)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
[023-030] searching : loss=0.98, accuracy@1=65.71%, accuracy@5=96.79%, time-cost=69027.4 s
[023-030] evaluate  : loss=0.91, accuracy@1=68.06%, accuracy@5=97.16%
<<<--->>> The 023-030-th epoch : Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find T1_K2_hess_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into T1_K2_hess_seed0/output/search

*SEARCH* [2021-02-08 09:13:06] [026-030][000/391] Time 6.33 (6.33) Data 0.30 (0.30) Arch [Loss 0.893 (0.893)  Prec@1 67.19 (67.19) Prec@5 92.19 (92.19)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 09:25:21] [026-030][100/391] Time 7.24 (7.34) Data 0.00 (0.00) Arch [Loss 0.949 (0.926)  Prec@1 67.19 (68.02) Prec@5 100.00 (96.97)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 09:37:41] [026-030][200/391] Time 6.31 (7.37) Data 0.00 (0.00) Arch [Loss 0.932 (0.927)  Prec@1 68.75 (67.99) Prec@5 96.88 (96.95)]
Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 09:50:00] [026-030][300/391] Time 7.65 (7.38) Data 0.00 (0.00) Arch [Loss 0.932 (0.932)  Prec@1 65.62 (67.77) Prec@5 96.88 (96.93)]
Structure(4 nodes with |nor

*SEARCH* [2021-02-08 11:14:43] [028-030][200/391] Time 7.32 (7.33) Data 0.00 (0.00) Arch [Loss 0.918 (0.903)  Prec@1 65.62 (69.02) Prec@5 100.00 (97.15)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 11:27:07] [028-030][300/391] Time 7.65 (7.37) Data 0.00 (0.00) Arch [Loss 1.002 (0.900)  Prec@1 62.50 (69.23) Prec@5 93.75 (97.29)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
*SEARCH* [2021-02-08 11:38:07] [028-030][390/391] Time 7.80 (7.36) Data 0.00 (0.00) Arch [Loss 0.795 (0.898)  Prec@1 72.50 (69.15) Prec@5 97.50 (97.24)]
Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|)
[028-030] searching : loss=0.90, accuracy@1=69.15%, accuracy@5=97.24%, time-cost=83524.8 s
[028-030] evaluate  : loss=0.88, accuracy@1=69.48%, accuracy@5=97.19%
<<<--->>> The 028-0